Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor 0.01 para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



<p style='text-align: justify'>
<ul>
    <li> Importando 'random' e algumas funções criadas, do arquivo 'funcoes.py', a serem usadas no algoritmo e as renomeando de maneira "auto-explicativa":</li>
</ul></p>

In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



<p style='text-align: justify'>
<ul>
    <li>Definindo as constantes, sendo elas: relacionadas à busca, que, se mudadas, alteram a eficácia do algoritmo; e relacionadas ao problema a ser resolvido, que, caso alteradas, mudam o problema em questão que se está resolvendo:</li>
</ul></p>

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15 # unidades de massa
OBJETOS = {
    # dicionário baseado no que enviamos na aula de Lógica, representando a mochila

    "Vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "Mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "Tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "Teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "Bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "Lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "Energético": {
        "peso": 2,
        "valor": 1500,
    },
    "Docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys())) # cria uma lista com os nomes das chaves ordenados em ordem 
                                               # alfabética

<p style='text-align: justify'>
<ul>
    <li> Definindo as funções locais, as quais utilizam de alguns valores padrões definidos neste notebook:</li>
</ul></p>

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

<p style='text-align: justify'>
<ul>
    <li> Código contendo a lógica para a resolução do problema:</li>
</ul></p>

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # determinando o melhor fitness, a priori, como menos infinito, 
                                        # visto que, apesar da restrição, se trata de maximização
melhor_individuo_ja_visto = [0] * NUM_OBJETOS # repete o número NUM_OBJETO vezes na lista sendo criada

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy() # copy() é utilizada para não ter o perigo de alterar o elemento
                                          # original em população também
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # entra no hall da fama só 
                                                                           # quem cumpre com os dois
                                                                           # pré-requisitos
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor indivíduo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES): # para cada dupla, 1 ou 0 e item:
    if pega_ou_nao == 1: # caso o gene indique 1
        print("+", item) # printa que o item correspondente deve ser pego
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
) # calcula e armazena, por fim, o valor e peso total da melhor opção de mochila que atende aos pedidos
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
) # printa este resultado

Maior valor: 13500 | Peso: 13
Maior valor: 16000 | Peso: 15

Você deve pegar os seguintes itens:
+ Energético
+ Lições em dia
+ Quadrinho super raro do Aranha-Homem da vida real
+ Vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 16000 dinheiros e peso de 15 unidades de massa.


**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.

Tentarei resolver depois! Bem como minhas sugestões abaixo.

## Conclusão

<p style='text-align: justify'> Com este algoritmo, foi possível encontrarmos solução para o problema da mochila. Relembrando, ele consiste em termos uma gama de objetos (10), com seus respectivos pesos e valores, disponíveis para escolhermos e devemos colocá-los na mochila de forma a maximizar o valor recolhido, porém tendo a restrição de não passar o peso máximo estabelecido (15 unidades de massa), caso contrário, a mochila rasga. Para que isso fosse possível, foram reutilizadas as funções: populacao_cb, selecao_roleta_max, cruzamento_ponto_simples e mutacao_cb. E foram adicionadas mais algumas funções: computa_mochila e funcao_objetivo_pop_mochila. Isso possibilitou a resolução, pois, após se ter cada nova população: o fitness para esse caso da mochila era calculado (ou sendo a soma dos valores de tudo que nela se encontra ou a penalidade aplicada) para toda a população; encotrava-se o maior valor dessa lista de fitness formada; era pego seu index/posição correspondente; ele era aplicado para pegar o respectivo indivíduo da população e armazenado em 'individuo' como cópia, para não apresentar problema de acabar sendo mudado na lista original da população; em seguida, calculava-se e armazenava-se o valor e o peso da mochila para esse indivíduo (associa a lista indivíduo que diz, por meio de 0 e 1, o que não se deve e o que se deve pegar, respectivamente, com o dicionário contendo as informações dos objetos e uma lista feita a partir dele com a ordem - no caso, alfabética - que devem ser analisados); caso esse maior fitnesse encontrado para a população em análise da vez fosse maior que outro visto anteriormente e o peso não excede o limite imposto, eles se tornavam o melhor fitness e o melhor indivíduo. Após repetir para todas as populações que o número de gerações permitia, e nos mostrar o resultado a cada nova melhor opção encontrada, o resultado final nos é mostrado, relacionando, novamente os genes 1 e 0 com a lista ordenada dos itens. </p>

<p style='text-align: justify'> Rodando diversas vezes, vemos que esse algoritmo se caracteriza como <b>probabilístico</b>, já que apresenta novos resultados a cada vez. Outro ponto interessante sobre isso é que este, assim como o problema anterior, também é do tipo NP difícil, logo, não há um algoritmo de total eficácia, a não ser que se testem todas as possibilidades</p>

<p style='text-align: justify'> Pensando no uso da equação f (que é uma função por partes, já que apresenta dois resultados diferentes dependendo de condições), caso não exista uma solução para um certo problema da mochila, por exemplo, se todos os itens excedam, por si só, o peso limite, ela sempre assumirá falor 0.01 e a solução vai ser não pegar nada, para esse exemplo.</p>

<p style='text-align: justify'> Ainda se tratando dessa função, ela confere a propriedade de penalizar, atribuindo o valor 0.01, o que não se cumpre com a restrição imposta pelo problema. Pensando em outras estratégias para penalisar soluções inválidas, acredito que seria interessante tentar eliminar tais indivíduos de solução inválida/indesejada e, por conseguinte, a criação de novos substituintes ou, então, algum tipo de operador que transforma os inválidos em válidos, como se fosse um mecanismo de revisão e reparo de DNA na biologia. Testes como esses vão ser anotados aqui para tentativa de implementação futura!</p>

<p style='text-align: justify'> Ademais, trazendo à tona as aulas de Lógica tidas, havia o problema da mochila considerando que se pode pegar diferentes porcentagens da quantidade disponível dos objetos para se completar e maximixar o problema da mochila, sendo os pesoas dados em relação a cada unidade da quantidade dos itens à disposição. Tentei iniciar a resolução de um problema desse tipo aqui, mas ainda não consegui terminar de implementar... em breve atualizo com a resolução! Mas minha estratégia seria alterar o modo como o indivíduo é computado, então, além de apenas se ter uma lista de 0 (não pega) e 1 (pega), deveria ser feita uma lista com números que poderiam variar de modo a indicar a porcentagem a ser pega e para poder se calcular, então, o valor respectivo a quantidade e o peso. A priori, conversando com o Daniel, pensamos que os genes poderiam variar com valores no intervalo [0, 1] (como 0.2, 0.5, 0.4, 0.85, 0.98), mas, pensando em seguida, daria pra aproveitar o que foi feito para o <a href="https://github.com/viyuetuki/aula_redes/blob/main/AlgoritmosGeneticos/experimento%20A.04%20-%20caixas%20nao-binarias.ipynb">problema das caixas não-binárias</a>, em que os genes variam e em um intervalo de 0 a 100.</p>

## Playground



### Desafio - problema da mochila considerando porcentagens de cada item

<i>Ainda tá bem incompleta a resolução... apenas estava importando as funções que pensei em usar para este notebook.</i>

## Importações

In [5]:
from funcoes import populacao_cnb
from funcoes import funcao_objetivo_pop_cnb as funcao_objetivo_pop
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cnb

## Código e Discussão

In [6]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15 # unidades de massa
OBJETOS = {

    "Diamante": {
        "peso disponível [Kg]": 2,
        "valor/Kg": 25000,
    },
    "Ouro": {
        "peso disponível [Kg]": 3,
        "valor/Kg": 15000,
    },
    "Granito": {
        "peso disponível [Kg]": 10,
        "valor/Kg": 7000,
    },
    "Prata": {
        "peso disponível [Kg]": 4,
        "valor/Kg": 1500,
    },
    "Esmeralda": {
        "peso disponível [Kg]": 6,
        "valor/Kg": 2400,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys())) #cria uma lista com os nomes das chaves ordenados em ordem alfabética

In [7]:
# funções locais

def cria_populacao_inicial(tamanho, numero_genes):
    return populacao_cnb(tamanho, numero_genes, VALOR_MAX_CAIXA)

def funcao_mutacao(individuo):
    return mutacao_cnb(individuo, VALOR_MAX_CAIXA)

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [8]:
# Busca por algoritmo genético

#populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
#melhor_fitness_ja_visto = -float("inf")
#melhor_individuo_ja_visto = [0] * NUM_OBJETOS # repete o número NUM_OBJETO vezes na lista sendo criada

#for n in range(NUM_GERACOES):

    # Seleção
#    fitness = funcao_objetivo_pop(populacao)
#    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
#    pais = populacao[0::2]
#    maes = populacao[1::2]

#    contador = 0

#    for pai, mae in zip(pais, maes):
#        if random.random() <= CHANCE_CRUZAMENTO:
#            filho1, filho2 = funcao_cruzamento(pai, mae)
#            populacao[contador] = filho1
#            populacao[contador + 1] = filho2

#        contador = contador + 2

    # Mutação
#    for n in range(len(populacao)):
#        if random.random() <= CHANCE_MUTACAO:
#            individuo = populacao[n]
#            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
#    fitness = funcao_objetivo_pop(populacao)
#    maior_fitness = max(fitness)
#    posicao = fitness.index(maior_fitness)
#    individuo = populacao[posicao].copy()
#    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
#    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # entra no hall da fama só quem cumpre com os pré-requisitos
#        melhor_fitness_ja_visto = maior_fitness
#        melhor_individuo_ja_visto = individuo
#        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
#print()
#print("Você deve pegar os seguintes itens:")
#for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
#    if pega_ou_nao == 1:
#        print("+", item)
#print()
#valor_total, peso_total = computa_mochila(
#    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
#)
#print(
#    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
#    f"e peso de {peso_total} unidades de massa."
#)